In [16]:
WORKING_DIR = "/home/xavier/Documents/DAE_project"

In [43]:
import os
import cv2
import numpy as np

image_dir = f"{WORKING_DIR}/images/sup_figure1/Original images"

images = []
for i in range(1, 7):
    img = cv2.imread(f"{image_dir}/Image{i}-1.jpg", cv2.IMREAD_ANYDEPTH)
    images.append(img)

In [30]:
# output_dir = f"{WORKING_DIR}/images/sup_figure1/Processed_for_Illustrator"
# os.makedirs(output_dir, exist_ok=True)

# for i, img in enumerate(images, 1):
#     # Convert 16-bit to 8-bit (simple division)
#     img_8bit = (img / 256).astype(np.uint8)
    
#     # Normalize to 0-255 range (contrast enhancement - recommended)
#     img_normalized = ((img.astype(np.float32) - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)
    
#     # Optional: Apply gamma correction for brightness adjustment
#     gamma = 0.8  # Adjust this value: <1 brightens, >1 darkens
#     img_gamma = np.power(img_normalized / 255.0, gamma)
#     img_gamma = (img_gamma * 255).astype(np.uint8)
    
#     cv2.imwrite(f"{output_dir}/Image{i}_8bit.png", img_8bit)
#     cv2.imwrite(f"{output_dir}/Image{i}_normalized.png", img_normalized)
#     cv2.imwrite(f"{output_dir}/Image{i}_gamma.png", img_gamma)
    
#     print(f"Image {i} saved")
#     print(f"  8-bit version: {img_8bit.min()}-{img_8bit.max()}")
#     print(f"  Normalized version: {img_normalized.min()}-{img_normalized.max()}")
#     print(f"  Gamma-corrected version: {img_gamma.min()}-{img_gamma.max()}")
#     print()

# print(f"All files saved to: {output_dir}")


Image 1 saved
  8-bit version: 0-7
  Normalized version: 0-255
  Gamma-corrected version: 0-255

Image 2 saved
  8-bit version: 0-7
  Normalized version: 0-255
  Gamma-corrected version: 0-255

Image 3 saved
  8-bit version: 0-6
  Normalized version: 0-255
  Gamma-corrected version: 0-255

Image 4 saved
  8-bit version: 0-6
  Normalized version: 0-255
  Gamma-corrected version: 0-255

Image 5 saved
  8-bit version: 0-7
  Normalized version: 0-255
  Gamma-corrected version: 0-255

Image 6 saved
  8-bit version: 0-13
  Normalized version: 0-255
  Gamma-corrected version: 0-255

All files saved to: /home/xavier/Documents/DAE_project/images/sup_figure1/Processed_for_Illustrator


In [51]:
# Extract multiple random cell crops from Image 1 - normalized version (square crops)
# Resolution: 0.65 micron/pixel
# Cell size: ~6 micron
# Crop size: 30x30 pixels (~19.5 micron) - square format with uniform size

resolution_micron_per_pixel = 0.65
cell_size_micron = 6
cell_size_pixels = int(cell_size_micron / resolution_micron_per_pixel)
crop_size = 40  # pixels - square crop size

print(f"Cell size: ~{cell_size_pixels} pixels")
print(f"Crop size: {crop_size}x{crop_size} pixels (square, ~{crop_size*resolution_micron_per_pixel:.1f} micron)\n")

# Get Image 1 and its normalized version
img1_original = images[0]

h, w = img1_original.shape
print(f"Image 1 size: {h}x{w} pixels = {h*resolution_micron_per_pixel:.1f}x{w*resolution_micron_per_pixel:.1f} micron\n")

# Generate random crop locations (avoiding edges)
np.random.seed(42)  # For reproducibility
margin = crop_size // 2 + 5  # Ensure enough margin for full square crop
num_crops = 6
upscale_factor = 3

valid_x = np.random.randint(margin, w - margin, num_crops)
valid_y = np.random.randint(margin, h - margin, num_crops)

print(f"Generating {num_crops} random square crops:\n")

# Save crops from each location
cell_output_dir = f"{WORKING_DIR}/images/sup_figure1/Cell_closeups"
os.makedirs(cell_output_dir, exist_ok=True)

for idx, (x_center, y_center) in enumerate(zip(valid_x, valid_y), 1):
    # Crop square region (guaranteed to be full size)
    x_start = x_center - crop_size // 2
    x_end = x_start + crop_size
    y_start = y_center - crop_size // 2
    y_end = y_start + crop_size
    
    # Ensure within bounds
    x_start = max(0, min(x_start, w - crop_size))
    x_end = x_start + crop_size
    y_start = max(0, min(y_start, h - crop_size))
    y_end = y_start + crop_size
    
    # Extract normalized crop (guaranteed square)
    cell_crop_norm = img1_original[y_start:y_end, x_start:x_end]
    
    # Verify it's square
    assert cell_crop_norm.shape == (crop_size, crop_size), f"Crop {idx} is not square!"
    
    # Save normalized version
    cv2.imwrite(f"{cell_output_dir}/Image1_cell_{idx:02d}.png", cell_crop_norm)
    
    original_h, original_w = cell_crop_norm.shape
    new_h = original_h * upscale_factor
    new_w = original_w * upscale_factor
    
    # Method 1: Lanczos4 interpolation (highest quality for upscaling)
    img_lanczos = cv2.resize(cell_crop_norm, (new_w, new_h), interpolation=cv2.INTER_LANCZOS4)
    
    # Method 2: Cubic interpolation (good balance of quality and speed)
    img_cubic = cv2.resize(cell_crop_norm, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    
    # Method 3: Lanczos4 + slight sharpening (enhances edges)
    img_lanczos_sharp = cv2.resize(cell_crop_norm, (new_w, new_h), interpolation=cv2.INTER_LANCZOS4)
    kernel_sharp = np.array([[-0.5, -1, -0.5],
                             [-1,   5, -1],
                             [-0.5, -1, -0.5]]) / 2.0
    img_lanczos_sharp = cv2.filter2D(img_lanczos_sharp, -1, kernel_sharp)
    img_lanczos_sharp = np.clip(img_lanczos_sharp, 0, 255).astype(np.uint8)
    
    # Save all versions
    cv2.imwrite(f"{cell_output_dir}/Image1_cell_{idx:02d}_upscale{upscale_factor}x_lanczos.png", img_lanczos)
    cv2.imwrite(f"{cell_output_dir}/Image1_cell_{idx:02d}_upscale{upscale_factor}x_cubic.png", img_cubic)
    cv2.imwrite(f"{cell_output_dir}/Image1_cell_{idx:02d}_upscale{upscale_factor}x_sharp.png", img_lanczos_sharp)
    
    intensity_avg = cell_crop_norm.mean()
    print(f"Cell {idx}: Center ({x_center}, {y_center}), Avg intensity: {intensity_avg:.1f}")
    print(f"  Crop size: {cell_crop_norm.shape[1]}x{cell_crop_norm.shape[0]} pixels (square)")
    print(f"  Size in microns: {crop_size*resolution_micron_per_pixel:.1f}x{crop_size*resolution_micron_per_pixel:.1f} micron")
    print()

print(f"✓ Saved {num_crops} random square cell crops (all {crop_size}x{crop_size} px) to: {cell_output_dir}")

# Create individual Image 1 with red box for each crop location
# Each cell crop has its own Image 1 copy showing only that cell's location
np.random.seed(42)
margin = crop_size // 2 + 5
valid_x_viz = np.random.randint(margin, w - margin, num_crops)
valid_y_viz = np.random.randint(margin, h - margin, num_crops)

# Draw red rectangles at each crop location
red_color = (0, 0, 255)  # BGR format: Red
line_thickness = 2

for idx, (x_center, y_center) in enumerate(zip(valid_x_viz, valid_y_viz), 1):
    # Create a fresh copy of the normalized image for each annotation
    img1_annotated = cv2.cvtColor(img1_original, cv2.COLOR_GRAY2BGR)
    
    # Calculate box corners for this crop
    x_start = x_center - crop_size // 2
    x_end = x_start + crop_size
    y_start = y_center - crop_size // 2
    y_end = y_start + crop_size
    
    # Ensure within bounds
    x_start = max(0, min(x_start, w - crop_size))
    x_end = x_start + crop_size
    y_start = max(0, min(y_start, h - crop_size))
    y_end = y_start + crop_size
    
    # Draw only this single rectangle (no label)
    cv2.rectangle(img1_annotated, (x_start, y_start), (x_end, y_end), red_color, line_thickness)
    
    # Save annotated image for this specific crop
    cv2.imwrite(f"{cell_output_dir}/Image1_cell_{idx:02d}_location.png", img1_annotated)

print(f"✓ Saved {num_crops} Image 1 copies, each with red box marking one crop location:")
for i in range(1, num_crops + 1):
    print(f"  Image1_cell_{i:02d}_location.png + Image1_cell_{i:02d}_normalized.png")



Cell size: ~9 pixels
Crop size: 40x40 pixels (square, ~26.0 micron)

Image 1 size: 400x400 pixels = 260.0x260.0 micron

Generating 6 random square crops:

Cell 1: Center (127, 45), Avg intensity: 97.2
  Crop size: 40x40 pixels (square)
  Size in microns: 26.0x26.0 micron

Cell 2: Center (373, 127), Avg intensity: 79.5
  Crop size: 40x40 pixels (square)
  Size in microns: 26.0x26.0 micron

Cell 3: Center (295, 146), Avg intensity: 56.4
  Crop size: 40x40 pixels (square)
  Size in microns: 26.0x26.0 micron

Cell 4: Center (131, 239), Avg intensity: 62.3
  Crop size: 40x40 pixels (square)
  Size in microns: 26.0x26.0 micron

Cell 5: Center (96, 355), Avg intensity: 76.3
  Crop size: 40x40 pixels (square)
  Size in microns: 26.0x26.0 micron

Cell 6: Center (213, 112), Avg intensity: 76.7
  Crop size: 40x40 pixels (square)
  Size in microns: 26.0x26.0 micron

✓ Saved 6 random square cell crops (all 40x40 px) to: /home/xavier/Documents/DAE_project/images/sup_figure1/Cell_closeups
✓ Saved 6 I